In [ ]:
PATH_MODULES = "../prototipos/"
exec(open(PATH_MODULES+"setup.py").read())
exec(open(PATH_MODULES+"datasetassembler.py").read())
exec(open(PATH_MODULES+"classify.py").read())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
ds = jsonToSklearn(PATH_DATAPOINTS+"notas_pandeiro.json")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ds["data"],\
                                                    ds["target"],\
                                                    test_size=1/3)

acc, model = KNNClassify(x_train, y_train, x_test, y_test, n_neighbors=1)
(acc, model)

(0.9, KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                      weights='uniform'))

In [ ]:
result = permutation_importance(model, x_test, y_test,
                                n_repeats=3000,
                                random_state=0) #Valor fixo para reprodutibilidade

In [ ]:
result.importances_mean

array([0.        , 0.        , 0.01448333, 0.07490833, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.05200833, 0.01940833, 0.02163333, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.27604167])

In [ ]:
for i in result.importances_mean.argsort()[::-1]:
  if result.importances_mean[i]>0: #- 2 * result.importances_std[i] > 0:
    print(ds["feature_names"][i],
          f"{result.importances_mean[i]:.3f}",
          f" +/- {result.importances_std[i]:.3f}")

strong_peak 0.276  +/- 0.069
spectral_centroid_time 0.075  +/- 0.027
spectral_peaks_freq_max 0.052  +/- 0.032
spectral_peaks_freq_median 0.022  +/- 0.018
spectral_peaks_freq_mean 0.019  +/- 0.017
max_mag_freq 0.014  +/- 0.018


###Teste com a melhor feature

In [ ]:
small = {}
small["data"] = [[point[-1]] for point in ds["data"]]
small["feature_names"] = ["strong_peak"]
small["target"] = ds["target"]
small["target_names"] = ds["target_names"]


In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(small["data"],\
                                                    small["target"],\
                                                    test_size=1/3)

acc2, model2 = KNNClassify(x_train2, y_train2, x_test2, y_test2, n_neighbors=1)
(acc2, model2)

(0.7, KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                      weights='uniform'))

In [ ]:
total2 = {}
qtd_modelos = 1000
for prop in range(50, 0, -5):
  estatisticas = KNNTestKs(dataset=small,
                           proportion_test=prop/100,
                           n=qtd_modelos)
  total2[str(prop/100)] = estatisticas
  print("prop = ", prop/100)
  print(sorted(estatisticas,key=lambda x: x["k"]))


prop =  0.5
[{'k': 1, 'mean': 0.7040666666666666, 'stdev': 0.05156451201067357}, {'k': 2, 'mean': 0.6682666666666667, 'stdev': 0.04984902130358152}, {'k': 3, 'mean': 0.70275, 'stdev': 0.051740303472707735}, {'k': 4, 'mean': 0.6935666666666667, 'stdev': 0.05684441885199714}, {'k': 5, 'mean': 0.7266333333333334, 'stdev': 0.05256303713516632}, {'k': 6, 'mean': 0.7036166666666667, 'stdev': 0.06024712528406549}, {'k': 7, 'mean': 0.7270666666666666, 'stdev': 0.05655906484999195}, {'k': 8, 'mean': 0.7022, 'stdev': 0.06386456863999018}, {'k': 9, 'mean': 0.7224833333333334, 'stdev': 0.06126152886897134}]
prop =  0.45
[{'k': 1, 'mean': 0.7086851851851852, 'stdev': 0.051972463940670593}, {'k': 2, 'mean': 0.6710925925925926, 'stdev': 0.05107044975620028}, {'k': 3, 'mean': 0.702462962962963, 'stdev': 0.04852904316998179}, {'k': 4, 'mean': 0.6975370370370371, 'stdev': 0.055515720589252276}, {'k': 5, 'mean': 0.7335925925925926, 'stdev': 0.05202648138918945}, {'k': 6, 'mean': 0.7091851851851851, 'stde

In [ ]:
print("ESTATÍSTICAS CALCULADAS PARA", qtd_modelos, "MODELOS\n")

print("Número de vizinhos com a maior acurácia média por proporção de teste para treino:")
max_means = []
for key, value in total2.items():
  means = [ (val["k"],val["mean"]) for val in value]
  max_mean = max(means, key=lambda k: k[1])
  max_means.append((key, max_mean))
  print("\t", key, ": n vizinhos =", max_mean[0], ", mean acurácia: ", max_mean[1])

print("\nMelhor:", max(max_means, key = lambda pair: pair[1][1]), "\n")

min_stdevs = []
print("Número de vizinhos com a menor desvio padrão de acurácia por proporção de teste para treino:")
for key, value in total2.items():
  stdevs = [ (val["k"],val["stdev"]) for val in value]
  min_stdev = min(stdevs, key=lambda k: k[1])
  min_stdevs.append((key, min_stdev))
  print("\t", key, ": n vizinhos =", min_stdev[0], ", stdev acurácia: ", min_stdev[1])

print("\nMelhor:", min(min_stdevs, key = lambda pair: pair[1][1]), "\n")

ESTATÍSTICAS CALCULADAS PARA 1000 MODELOS

Número de vizinhos com a maior acurácia média por proporção de teste para treino:
	 0.5 : n vizinhos = 7 , mean acurácia:  0.7270666666666666
	 0.45 : n vizinhos = 7 , mean acurácia:  0.7342407407407407
	 0.4 : n vizinhos = 7 , mean acurácia:  0.7382291666666667
	 0.35 : n vizinhos = 7 , mean acurácia:  0.7420714285714286
	 0.3 : n vizinhos = 7 , mean acurácia:  0.7437777777777778
	 0.25 : n vizinhos = 7 , mean acurácia:  0.7455666666666667
	 0.2 : n vizinhos = 9 , mean acurácia:  0.7462083333333334
	 0.15 : n vizinhos = 9 , mean acurácia:  0.7503888888888889
	 0.1 : n vizinhos = 9 , mean acurácia:  0.7568333333333334
	 0.05 : n vizinhos = 9 , mean acurácia:  0.7588333333333334

Melhor: ('0.05', (9, 0.7588333333333334)) 

Número de vizinhos com a menor desvio padrão de acurácia por proporção de teste para treino:
	 0.5 : n vizinhos = 2 , stdev acurácia:  0.04984902130358152
	 0.45 : n vizinhos = 3 , stdev acurácia:  0.04852904316998179
	 0.4 :

###Teste com todas as features boas

In [ ]:
good_features = ["strong_peak",
"spectral_centroid_time",
"spectral_peaks_freq_max",
"spectral_peaks_freq_median",
"spectral_peaks_freq_mean",
"max_mag_freq"]

g_index2feat = []

for g in good_features:
  for i in range(0,len(ds["feature_names"])):
    if g in ds["feature_names"][i]:
      g_index2feat.append((i,g))

g_index2feat

[(30, 'strong_peak'),
 (3, 'spectral_centroid_time'),
 (10, 'spectral_peaks_freq_max'),
 (12, 'spectral_peaks_freq_median'),
 (11, 'spectral_peaks_freq_mean'),
 (2, 'max_mag_freq')]

In [ ]:
#[p[30],p[3],p[10],p[12],p[11],p[2]]


medium = {}


medium["data"] = [[p[30],p[3],p[10],p[12],p[11],p[2]] for p in ds["data"]]
medium["feature_names"] = good_features
medium["target"] = ds["target"]
medium["target_names"] = ds["target_names"]

In [ ]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(medium["data"],\
                                                    medium["target"],\
                                                    test_size=1/3)
acc3, model3 = KNNClassify(x_train3, y_train3, x_test3, y_test3, n_neighbors=1)
(acc3, model3)

(0.95, KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                      weights='uniform'))

In [ ]:
total3 = {}
qtd_modelos = 1000
for prop in range(50, 0, -5):
  estatisticas = KNNTestKs(dataset=medium,
                           proportion_test=prop/100,
                           n=qtd_modelos)
  total3[str(prop/100)] = estatisticas
  print("prop = ", prop/100)
  print(sorted(estatisticas,key=lambda x: x["k"]))


prop =  0.5
[{'k': 1, 'mean': 0.9104333333333333, 'stdev': 0.033961891536594506}, {'k': 2, 'mean': 0.88095, 'stdev': 0.039321033094816354}, {'k': 3, 'mean': 0.8966833333333333, 'stdev': 0.03641646900470068}, {'k': 4, 'mean': 0.8872833333333333, 'stdev': 0.04655532441692247}, {'k': 5, 'mean': 0.8871666666666667, 'stdev': 0.04874438330508122}, {'k': 6, 'mean': 0.8651333333333333, 'stdev': 0.05793583524425874}, {'k': 7, 'mean': 0.8634666666666667, 'stdev': 0.057457764732552846}, {'k': 8, 'mean': 0.8375833333333333, 'stdev': 0.06281848007725907}, {'k': 9, 'mean': 0.8377, 'stdev': 0.06259095483875947}]
prop =  0.45
[{'k': 1, 'mean': 0.9164444444444444, 'stdev': 0.03446295460086729}, {'k': 2, 'mean': 0.882537037037037, 'stdev': 0.03887900561696035}, {'k': 3, 'mean': 0.8998333333333334, 'stdev': 0.036672467931647995}, {'k': 4, 'mean': 0.8953148148148148, 'stdev': 0.042712837154518055}, {'k': 5, 'mean': 0.8957962962962963, 'stdev': 0.0433798355991725}, {'k': 6, 'mean': 0.8798518518518519, 'std

In [ ]:
print("ESTATÍSTICAS CALCULADAS PARA", qtd_modelos, "MODELOS\n")

print("Número de vizinhos com a maior acurácia média por proporção de teste para treino:")
max_means = []
for key, value in total3.items():
  means = [ (val["k"],val["mean"]) for val in value]
  max_mean = max(means, key=lambda k: k[1])
  max_means.append((key, max_mean))
  print("\t", key, ": n vizinhos =", max_mean[0], ", mean acurácia: ", max_mean[1])

print("\nMelhor:", max(max_means, key = lambda pair: pair[1][1]), "\n")

min_stdevs = []
print("Número de vizinhos com a menor desvio padrão de acurácia por proporção de teste para treino:")
for key, value in total3.items():
  stdevs = [ (val["k"],val["stdev"]) for val in value]
  min_stdev = min(stdevs, key=lambda k: k[1])
  min_stdevs.append((key, min_stdev))
  print("\t", key, ": n vizinhos =", min_stdev[0], ", stdev acurácia: ", min_stdev[1])

print("\nMelhor:", min(min_stdevs, key = lambda pair: pair[1][1]), "\n")

ESTATÍSTICAS CALCULADAS PARA 1000 MODELOS

Número de vizinhos com a maior acurácia média por proporção de teste para treino:
	 0.5 : n vizinhos = 1 , mean acurácia:  0.9104333333333333
	 0.45 : n vizinhos = 1 , mean acurácia:  0.9164444444444444
	 0.4 : n vizinhos = 1 , mean acurácia:  0.917125
	 0.35 : n vizinhos = 1 , mean acurácia:  0.9224761904761904
	 0.3 : n vizinhos = 1 , mean acurácia:  0.9278333333333333
	 0.25 : n vizinhos = 1 , mean acurácia:  0.9282333333333334
	 0.2 : n vizinhos = 1 , mean acurácia:  0.9310833333333334
	 0.15 : n vizinhos = 1 , mean acurácia:  0.9354444444444444
	 0.1 : n vizinhos = 1 , mean acurácia:  0.9351666666666667
	 0.05 : n vizinhos = 1 , mean acurácia:  0.9430000000000001

Melhor: ('0.05', (1, 0.9430000000000001)) 

Número de vizinhos com a menor desvio padrão de acurácia por proporção de teste para treino:
	 0.5 : n vizinhos = 1 , stdev acurácia:  0.033961891536594506
	 0.45 : n vizinhos = 1 , stdev acurácia:  0.03446295460086729
	 0.4 : n vizinh